In [1]:
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
from config import username, password

In [2]:
# Read in race stats csv and convert it to a dataframe
csv_file = "data/2019_Ironman_World_Championship_Results.csv"
race_stats_df = pd.read_csv(csv_file)
race_stats_df.head()

,BIB,Last_Name,First_Name,Country,Gender,Division,Swim,Bike,Run,Overall,Division_Rank,Gender_Rank,Overall_Rank,T1,T2
0,4,Frodeno,Jan,DEU,Male,MPRO,0:47:31,4:16:02,2:42:43,7:51:13,1,1,1,0:01:58,0:02:59
1,6,O'donnell,Tim,USA,Male,MPRO,0:47:38,4:18:11,2:49:45,7:59:41,2,2,2,0:02:01,0:02:06
2,5,Kienle,Sebastian,DEU,Male,MPRO,0:52:17,4:15:04,2:49:56,8:02:04,3,3,3,0:02:12,0:02:35
3,14,Hoffman,Ben,USA,Male,MPRO,0:51:01,4:24:01,2:43:08,8:02:52,4,4,4,0:02:25,0:02:17
4,11,Wurf,Cameron,AUS,Male,MPRO,0:52:25,4:14:44,2:55:03,8:06:41,5,5,5,0:02:07,0:02:22


In [3]:
# Read in countries csv and convert it to a dataframe
csv_file = "data/countries_codes_and_coordinates.csv"
countries_df = pd.read_csv(csv_file)
countries_df.head()

,Country,Alpha_2_code,Alpha_3_code,Numeric_code,Latitude_average,Longitude_average
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6


In [13]:
merged_df = pd.merge(race_stats_df, countries_df, left_on="Country", right_on="Alpha_3_code")
merged_df.head()

,BIB,Last_Name,First_Name,Country_x,Gender,Division,Swim,Bike,Run,Overall,...,Gender_Rank,Overall_Rank,T1,T2,Country_y,Alpha_2_code,Alpha_3_code,Numeric_code,Latitude_average,Longitude_average
0,4,Frodeno,Jan,DEU,Male,MPRO,0:47:31,4:16:02,2:42:43,7:51:13,...,1,1,0:01:58,0:02:59,Germany,DE,DEU,276,51.0,9.0
1,5,Kienle,Sebastian,DEU,Male,MPRO,0:52:17,4:15:04,2:49:56,8:02:04,...,3,3,0:02:12,0:02:35,Germany,DE,DEU,276,51.0,9.0
2,45,Frommhold,Nils,DEU,Male,MPRO,0:50:47,4:24:29,3:05:10,8:24:56,...,20,20,0:02:08,0:02:22,Germany,DE,DEU,276,51.0,9.0
3,44,Duelsen,Marc,DEU,Male,MPRO,0:52:21,4:28:26,3:01:14,8:26:35,...,23,23,0:02:07,0:02:27,Germany,DE,DEU,276,51.0,9.0
4,3003,Haug,Anne,DEU,Female,FPRO,0:54:09,4:50:17,2:51:07,8:40:10,...,1,32,0:02:02,0:02:35,Germany,DE,DEU,276,51.0,9.0


In [14]:
merged_df.columns

Index(['BIB', 'Last_Name', 'First_Name', 'Country_x', 'Gender', 'Division',
       'Swim', 'Bike', 'Run', 'Overall', 'Division_Rank', 'Gender_Rank',
       'Overall_Rank', 'T1', 'T2', 'Country_y', 'Alpha_2_code', 'Alpha_3_code',
       'Numeric_code', 'Latitude_average', 'Longitude_average'],
      dtype='object')

In [15]:
merged_df = merged_df[['BIB', 'Last_Name', 'First_Name', 'Country_x', 'Country_y', 'Gender', 'Division', 'Swim', 'Bike', 'Run', 'Overall', 'Division_Rank', 'Gender_Rank', 'Overall_Rank', 'T1', 'T2', 'Latitude_average', 'Longitude_average']]
merged_df = merged_df.rename(columns={"Country_x":"Alpha_3_code", "Country_y":"Country"})
merged_df.head()

,BIB,Last_Name,First_Name,Alpha_3_code,Country,Gender,Division,Swim,Bike,Run,Overall,Division_Rank,Gender_Rank,Overall_Rank,T1,T2,Latitude_average,Longitude_average
0,4,Frodeno,Jan,DEU,Germany,Male,MPRO,0:47:31,4:16:02,2:42:43,7:51:13,1,1,1,0:01:58,0:02:59,51.0,9.0
1,5,Kienle,Sebastian,DEU,Germany,Male,MPRO,0:52:17,4:15:04,2:49:56,8:02:04,3,3,3,0:02:12,0:02:35,51.0,9.0
2,45,Frommhold,Nils,DEU,Germany,Male,MPRO,0:50:47,4:24:29,3:05:10,8:24:56,20,20,20,0:02:08,0:02:22,51.0,9.0
3,44,Duelsen,Marc,DEU,Germany,Male,MPRO,0:52:21,4:28:26,3:01:14,8:26:35,23,23,23,0:02:07,0:02:27,51.0,9.0
4,3003,Haug,Anne,DEU,Germany,Female,FPRO,0:54:09,4:50:17,2:51:07,8:40:10,1,1,32,0:02:02,0:02:35,51.0,9.0


In [16]:
# Create connection to postgresql
rds_connection_string = f"{username}:{password}@localhost:5432/ironman"
# save this to sqllite
engine = create_engine("sqlite:///data/ironman.sqlite")

In [17]:
# Use pandas to load countries DataFrame into database
merged_df.to_sql(name='race_stats', con=engine, if_exists='replace', index=False)

In [18]:
# verify table was loaded successfully
pd.read_sql_query('select * from race_stats', con=engine).head()

,BIB,Last_Name,First_Name,Alpha_3_code,Country,Gender,Division,Swim,Bike,Run,Overall,Division_Rank,Gender_Rank,Overall_Rank,T1,T2,Latitude_average,Longitude_average
0,4,Frodeno,Jan,DEU,Germany,Male,MPRO,0:47:31,4:16:02,2:42:43,7:51:13,1,1,1,0:01:58,0:02:59,51.0,9.0
1,5,Kienle,Sebastian,DEU,Germany,Male,MPRO,0:52:17,4:15:04,2:49:56,8:02:04,3,3,3,0:02:12,0:02:35,51.0,9.0
2,45,Frommhold,Nils,DEU,Germany,Male,MPRO,0:50:47,4:24:29,3:05:10,8:24:56,20,20,20,0:02:08,0:02:22,51.0,9.0
3,44,Duelsen,Marc,DEU,Germany,Male,MPRO,0:52:21,4:28:26,3:01:14,8:26:35,23,23,23,0:02:07,0:02:27,51.0,9.0
4,3003,Haug,Anne,DEU,Germany,Female,FPRO,0:54:09,4:50:17,2:51:07,8:40:10,1,1,32,0:02:02,0:02:35,51.0,9.0


In [19]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [20]:
engine = create_engine("sqlite:///data/ironman.sqlite")

In [21]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [22]:
Base.classes.keys()

[]